In [164]:
#importing libraries
import pandas as pd
import pickle
from pandas import DataFrame, Series
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model, naive_bayes
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score
import pybaseball as pyb
from pybaseball import statcast
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier


In [88]:
df = pd.read_csv("/Users/johndavis/Desktop/MLB_RV_100.csv",low_memory=False)

In [ ]:
df.head(1)

In [90]:
#Drop all rows with NaN values
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()
df = df.reset_index()

In [ ]:
df.head(1)

In [92]:
#create a new dataframe with only pitches that are fastballs, sinkers and cutters
dfb = df[(df['Pitch Type'] == 'FF') | (df['Pitch Type'] == 'FT') | (df['Pitch Type'] == 'SI') | (df['Pitch Type'] == 'FC')]

In [93]:
#create a column for the absolute value of the horizontal break
dfb.insert(6, "ABS Horizontal Break", abs(dfb["Horizontal Break"]), True)

In [94]:
#create a column for the differential between the vertical and horizontal break
dfb.insert(7, "Vertical - Horizontal Break", dfb["Vertical Break"] - dfb["Horizontal Break"], True)

In [ ]:
#rename Vertical - Horizontal Break to differential break
dfb.rename (columns = {'Vertical - Horizontal Break':'Differential Break'}, inplace = True)

In [ ]:
dfb.head(1)

In [97]:
dfb = dfb[['Player Name','Velocity','Pitch Type','Spin Rate','Vertical Break','Horizontal Break','Differential Break','Release Height','Side Release','Extension','Run Values']]

In [ ]:
dfb.head(1)


In [99]:
#Rename PLAYER NAME to PITCHER And Run Values to RV
dfb.rename (columns = {'Player Name':'PITCHER','Run Values':'RV'}, inplace = True)

In [100]:
X = dfb[['Velocity','Spin Rate','Vertical Break', 'Horizontal Break','Differential Break','Release Height','Side Release','Extension']]
y = dfb['RV'] 

In [101]:
#Split data into training and testing sets
train_test_split(X,y,test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [17]:
#Create a random forest regressor
rfr = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0)
rfr.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, random_state=0)

In [ ]:
#create a dataframe with the feature importances
feature_importances = pd.DataFrame(rfr.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances

In [103]:
#create a new column in dfb with the predicted run values
dfb['Predicted RV'] = rfr.predict(X)


In [104]:
#Create a new dataframe with averages of all the values 
dfb_avg = dfb.groupby(['PITCHER']).mean().reset_index()

In [105]:
#order the dataframe by the predicted run values lowest to highest 
dfb_avg = dfb_avg.sort_values(by=['Predicted RV'], ascending=True)

In [106]:
#check the dataframe
dfb_avg.head(5)

,PITCHER,Velocity,Spin Rate,Vertical Break,Horizontal Break,Differential Break,Release Height,Side Release,Extension,RV,Predicted RV
219,"Fairbanks, Pete",98.938710,2510.345622,20.527189,-0.209585,20.736774,6.986406,0.273502,6.617972,-0.014180,-0.013816
377,"Karinchak, James",95.219474,2354.389474,20.709158,-2.007158,22.716316,6.712158,0.200421,6.064474,-0.012326,-0.010473
282,"Glasnow, Tyler",98.052000,2476.880000,19.612800,-2.073600,21.686400,6.236400,1.677600,7.336000,0.041538,-0.007887
488,"Means, John",91.980723,2414.048193,20.489639,8.239518,12.250120,6.561687,0.899639,6.091566,-0.002592,-0.006128
232,"Feyereisen, J.P.",92.241176,2345.130719,20.205490,-3.914510,24.120000,6.508562,0.247124,5.836601,-0.047469,-0.006114


In [107]:
#Edit The Pitcher Names so First name is first and last name is last
dfb_avg['PITCHER'] = dfb_avg['PITCHER'].str.split(' ', expand=True)[1] + ' ' + dfb_avg['PITCHER'].str.split(' ', expand=True)[0]

In [108]:
#Remove the comma from the last name
dfb_avg = dfb_avg.replace(',','', regex=True)

In [109]:
#create a new column with predicted run value times 100 called xRV/100
dfb_avg['xRV/100'] = dfb_avg['Predicted RV'] * 100

In [110]:
#reset the index
dfb_avg = dfb_avg.reset_index()

In [111]:
#drop the index column
dfb_avg = dfb_avg.drop(['index'], axis=1)

In [ ]:
dfb_avg = dfb_avg[['PITCHER','Velocity','Spin Rate','Vertical Break','Horizontal Break','xRV/100']]
dfb_avg.head(5)

In [113]:
#Edit Horizontal Break to be absolute value of horizontal break
dfb_avg['Horizontal Break'] = abs(dfb_avg['Horizontal Break'])

In [114]:
dfb_avg.head(5)

,PITCHER,Velocity,Spin Rate,Vertical Break,Horizontal Break,xRV/100
0,Pete Fairbanks,98.938710,2510.345622,20.527189,0.209585,-1.381639
1,James Karinchak,95.219474,2354.389474,20.709158,2.007158,-1.047280
2,Tyler Glasnow,98.052000,2476.880000,19.612800,2.073600,-0.788680
3,John Means,91.980723,2414.048193,20.489639,8.239518,-0.612757
4,J.P. Feyereisen,92.241176,2345.130719,20.205490,3.914510,-0.611380


In [122]:
#add in a new column called pitch count that is the number of pitches thrown by each pitcher
dfb_avg = dfb_avg.assign(Pitch_Count=dfb.groupby('PITCHER').size().values)

In [128]:
#reorder the columns so pitch count is second
dfb_avg = dfb_avg[['Pitcher','Pitch_Count','Velocity','Spin Rate','Vertical Break','Horizontal Break','xRV/100']]

In [124]:
#change PITCHER to pitcher 
dfb_avg = dfb_avg.rename(columns={'PITCHER': 'Pitcher'})

In [125]:
#change pitcher to Pitcher 
dfb_avg = dfb_avg.rename(columns={'pitcher': 'Pitcher'})

In [ ]:
#find the mean of xRV/100
dfb_avg['xRV/100'].mean()

In [ ]:
dfb_avg.head(5)


In [131]:
#import dataframe from baseball refernce with Basic Pitcher Stats
df_baseball_ref = pd.read_csv("/Users/johndavis/Desktop/Basic_Stats.csv",low_memory=False, encoding='latin-1')


In [132]:
#Remove Ê FROM PITCHER NAMES AND REPLACE WITH A SPACE
df_baseball_ref['Name'] = df_baseball_ref['Name'].str.replace('Ê', ' ')

In [133]:
#take the average stats for each pitcher 
df_baseball_ref_avg = df_baseball_ref.groupby(['Name']).mean().reset_index()

In [134]:
#remove asterisks from pitcher names
df_baseball_ref_avg = df_baseball_ref_avg.replace('\*','', regex=True)

In [135]:
#change Name column to Pitcher
df_baseball_ref_avg = df_baseball_ref_avg.rename(columns={'Name': 'Pitcher'})

In [ ]:
df_baseball_ref_avg.head(5)

In [ ]:
#add ERA from df_baseball_ref_avg to dfb_avg that corresponds to the pitcher
dfb_avg = dfb_avg.merge(df_baseball_ref_avg[['Pitcher','ERA']], on='Pitcher', how='left')
dfb_avg.head(10)

In [140]:
#Reorder columns so Pitcher, Pitch_Count, Velocity, Spin Rate, Vertical Break, Horizontal Break, xRV/100, ERA
dfb_avg = dfb_avg[['Pitcher','Pitch_Count','Velocity','Spin Rate','Vertical Break','Horizontal Break','xRV/100','ERA']]


In [142]:
# get rid of pitchers with less than 100 pitches thrown
dfb_avg = dfb_avg[dfb_avg['Pitch_Count'] > 100]

In [143]:
#get rid of all Nan values
dfb_avg = dfb_avg.dropna()
dfb_avg = dfb_avg.reset_index()

In [146]:
#add in FIP column from df_baseball_ref_avg that corresponds to the pitcher
dfb_avg = dfb_avg.merge(df_baseball_ref_avg[['Pitcher','FIP']], on='Pitcher', how='left')


In [147]:
dfb_avg = dfb_avg[['Pitcher','Pitch_Count','Velocity','Spin Rate','Vertical Break','Horizontal Break','xRV/100','ERA','FIP']]

In [ ]:
dfb_avg.corr()

In [ ]:
#create a heatmap of avaerage velocity and xRV/100
g = sns.jointplot(x="Velocity", y="xRV/100", data=dfb_avg, kind="reg", truncate=False, color="m", height=7)
g.fig.suptitle("Average FB Velocity vs. xRV/100")
g.fig.subplots_adjust(top=.9)




In [ ]:
#create a visualation of Top 10 Pitchers with the highest xRV/100
p = sns.barplot(x="xRV/100", y="Pitcher", data=dfb_avg.head(10), palette= 'deep')
#p.set(xlabel='xRV/100', ylabel='Pitcher')
p.set_title('Top 10 Pitchers with Top FB xRV/100')